In [137]:
!pip install tensorflow-gpu==2.8.0
import tensorflow as tf

!pip install tensorflow-io==0.25.0

In [138]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [139]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

In [140]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [115]:
testing_wav_file_name = tf.keras.utils.get_file('all data.zip',origin="/kaggle/input/new-one",
                                                cache_dir='./',
                                                cache_subdir='datasets',
                                                extract=True)

print(testing_wav_file_name)

./datasets/all data.zip


In [161]:
annonations_files = '/kaggle/input/new-one/train_mapping.csv'
audio_dir_path = '/kaggle/input/new-one/training_split/'

pd_data = pd.read_csv(annonations_files)
pd_data.head()


,File_name,Person_id
0,0_19_male/0_19_male_0.wav,0
1,0_19_male/0_19_male_1.wav,0
2,0_19_male/0_19_male_2.wav,0
3,0_19_male/0_19_male_3.wav,0
4,0_19_male/0_19_male_4.wav,0


In [117]:
data={'File_name':['/kaggle/input/new-one/training_split/0_19_male/0_19_male_0.wav',
                   '/kaggle/input/new-one/training_split/10_21_female/10_21_female_0.wav'],
      'Person_id':[0,0]}

df = pd.DataFrame(data)

In [142]:
import torchaudio
import torch

In [143]:
target_sample_rate = 16000
num_samples = 16000

In [144]:
def _cut_if_necessary(signal):
        if signal.shape[1] > num_samples:
            signal = signal[:, :num_samples]
        return signal

def _right_pad_if_necessary(signal):
        length_signal = signal.shape[1]
        if length_signal < num_samples:
            num_missing_samples = num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

def _resample_if_necessary(signal, sr):
        if sr !=target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr,target_sample_rate)
            signal = resampler(signal)
        return signal

def _mix_down_if_necessary(signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

In [145]:
#@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    # file_contents = tf.io.read_file(filename)
    # wav, sample_rate = tf.audio.decode_wav(
    #       file_contents,
    #       desired_channels=1)
    # wav = tf.squeeze(wav, axis=-1)
    # sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)







    signal, sr = torchaudio.load(filename)
    signal =_resample_if_necessary(signal, sr)
    signal =_mix_down_if_necessary(signal)
    signal =_cut_if_necessary(signal)
    signal =_right_pad_if_necessary(signal)
    return signal.numpy().reshape(-1,)
    # return wav

In [146]:
def tensor_to_numpy(tensor):
    return tensor.numpy()

In [123]:
# import tensorflow as tf
# import tensorflow_io as tfio
# from scipy.io.wavfile import read

# def load_wav_16k_mono(filename_tensor):
#     """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
#     def load_wav(filename):
#         # Read the WAV file using scipy
#         sample_rate, wav_data = read(filename)

#         # Convert to float32
#         wav = tf.convert_to_tensor(wav_data, dtype=tf.float32)

#         # Resample to 16 kHz
#         wav = tfio.audio.resample(tf.reshape(wav, [1, -1]), rate_in=sample_rate, rate_out=16000)
#         wav = tf.squeeze(wav)  # Remove the unnecessary dimension

#         # If necessary, trim the waveform to 16000 samples
#         if wav.shape[0] > 16000:
#             wav = wav[:16000]

#         return wav

#     # Ensure that filename_tensor is a string tensor
#     filename_tensor = tf.convert_to_tensor(filename_tensor, dtype=tf.string)

#     # Use tf.numpy_function to enable eager execution within the function
#     wav = tf.numpy_function(load_wav, [filename_tensor], tf.float32)

#     return wav


In [148]:
testing_wav_data = load_wav_16k_mono('/kaggle/input/new-one/training_split/0_19_male/0_19_male_0.wav')
#_ = plt.plot(testing_wav_data)

In [149]:
type(testing_wav_data)

numpy.ndarray

In [150]:
print("Input Shape:", testing_wav_data.shape)


Input Shape: (16000,)


In [151]:
scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
class_scores = tf.reduce_mean(scores, axis=0)
top_class = tf.argmax(class_scores)


print(type(embeddings))
print(f'The embeddings shape: {embeddings.shape}')

<class 'tensorflow.python.framework.ops.EagerTensor'>
The embeddings shape: (2, 1024)


In [154]:
filenames = pd_data['File_name']
targets = pd_data['Person_id']

 #main_ds = tf.data.Dataset.from_tensor_slices((audio_dir_path+filenames, targets))

In [155]:
filenames

0               0_19_male/0_19_male_0.wav
1               0_19_male/0_19_male_1.wav
2               0_19_male/0_19_male_2.wav
3               0_19_male/0_19_male_3.wav
4               0_19_male/0_19_male_4.wav
                      ...                
1448    200_37_female/200_37_female_0.wav
1449    200_37_female/200_37_female_1.wav
1450    200_37_female/200_37_female_2.wav
1451    200_37_female/200_37_female_3.wav
1452    200_37_female/200_37_female_4.wav
Name: File_name, Length: 1453, dtype: object

In [156]:
type(filenames)

pandas.core.series.Series

In [162]:
def load_wav_for_map(filename):
  return load_wav_16k_mono(filename)

new_values=[]
for value in filenames:
  new_values.append(load_wav_for_map(audio_dir_path+value))

pd_data['File_name']=new_values

filenames = [str(filename) for filename in filenames]

# Convert the new_values list of NumPy arrays to a NumPy array
new_values_np = np.array(new_values)

# Create TensorFlow dataset
main_ds = tf.data.Dataset.from_tensor_slices((new_values_np, targets))


In [163]:
main_ds

<TensorSliceDataset element_spec=(TensorSpec(shape=(16000,), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [164]:
def extract_embedding(wav_data, label):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings)[0]
  return (embeddings,
            tf.repeat(label, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding) #.unbatch()
main_ds.element_spec

(TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [165]:
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(201)
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 dense_7 (Dense)             (None, 201)               103113    
                                                                 
Total params: 627,913
Trainable params: 627,913
Non-trainable params: 0
_________________________________________________________________


In [166]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [167]:
history = my_model.fit(main_ds,
                       epochs=20,
                       validation_data=main_ds,
                       callbacks=callback)

Epoch 1/20
1453/1453 [==============================] - 76s 52ms/step - loss: 5.7544 - accuracy: 0.0127 - val_loss: 5.5332 - val_accuracy: 0.0145
Epoch 2/20
1453/1453 [==============================] - 64s 44ms/step - loss: 4.3485 - accuracy: 0.0323 - val_loss: 5.0190 - val_accuracy: 0.0520
Epoch 3/20
1453/1453 [==============================] - 65s 44ms/step - loss: 3.6488 - accuracy: 0.0929 - val_loss: 4.9512 - val_accuracy: 0.0812
Epoch 4/20
1453/1453 [==============================] - 65s 44ms/step - loss: 3.1065 - accuracy: 0.1696 - val_loss: 5.3845 - val_accuracy: 0.1050
Epoch 5/20
1453/1453 [==============================] - 76s 52ms/step - loss: 2.6321 - accuracy: 0.2567 - val_loss: 6.0995 - val_accuracy: 0.1050
Epoch 6/20
1453/1453 [==============================] - 65s 44ms/step - loss: 2.3446 - accuracy: 0.3255 - val_loss: 6.0555 - val_accuracy: 0.1287
Epoch 7/20
1453/1453 [==============================] - 76s 52ms/step - loss: 2.0561 - accuracy: 0.4119 - val_loss: 5.9480 -